In [4]:
import torch
print(torch.cuda.is_available())

True


In [5]:
# 1. 필수 라이브러리 설치
# transformers: 모델 및 토크나이저 로드
# datasets: 데이터셋 다운로드 및 처리
# evaluate: 성능 평가 (Accuracy, F1 등)
# peft: 향후 사용할 경량화 튜닝 라이브러리
# accelerate: 학습 가속화 도구
!pip install -q transformers datasets evaluate peft accelerate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00


In [6]:
import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [8]:
from datasets import load_dataset

# 1. 설정 (Config)
MODEL_ID = "klue/roberta-base" # 한국어 성능이 좋은 RoBERTa 모델 사용

# 2. 데이터셋 로드
print("Loading dataset: klue, ynat...")
# 'klue'는 데이터셋 이름, 'ynat'는 그 안의 세부 주제(Task) 이름입니다.
dataset = load_dataset("klue", "ynat")

# 데이터 확인 (선택 사항)
print(f"Train data size: {len(dataset['train'])}")
print(f"Example: {dataset['train'][0]}")

Loading dataset: klue, ynat...


README.md: 0.00B [00:00, ?B/s]

ynat/train-00000-of-00001.parquet:   0%|          | 0.00/4.17M [00:00<?, ?B/s]

ynat/validation-00000-of-00001.parquet:   0%|          | 0.00/847k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

Train data size: 45678
Example: {'guid': 'ynat-v1_train_00000', 'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영', 'label': 3, 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947', 'date': '2016.06.30. 오전 10:36'}


In [13]:
# 3. 토크나이저 로드 및 전처리 함수 정의
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def preprocess_function(examples):
    # 제목(title)을 입력으로 받아 토큰화
    return tokenizer(examples["title"], truncation=True, padding=True)

# 4. 전체 데이터셋에 전처리 적용 (Map)
print("Tokenizing data...")
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels") # label -> labels 로 변경
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# 데이터 분류 (Train / Validation) - ynat은 validation이 이미 나뉘어 있음
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]

# Data Collator (배치 단위로 패딩 맞춤)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print("Data preparation complete.")

Tokenizing data...


Map:   0%|          | 0/45678 [00:00<?, ? examples/s]

Map:   0%|          | 0/9107 [00:00<?, ? examples/s]

Data preparation complete.


In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate

# 1. 모델 로드
# klue/ynat은 라벨이 7개입니다. (num_labels=7)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID, num_labels=7)

# ---------------------------------------------------------
# [핵심] Feature Extraction을 위해 Backbone 파라미터 얼리기 (Freeze)
# ---------------------------------------------------------
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# 얼려졌는지 확인 (학습 가능한 파라미터 수 출력)
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

print(">>> Feature Extraction Mode (Backbone Freezed)")
print_trainable_parameters(model)
# 결과 예시: trainable%가 1% 미만이어야 정상 (분류기 층만 학습하므로)

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


>>> Feature Extraction Mode (Backbone Freezed)
trainable params: 595975 || all params: 110623495 || trainable%: 0.54


In [15]:
# 2. 평가 지표 정의 (Accuracy, F1)
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    f1_score = f1.compute(predictions=predictions, references=labels, average="macro")
    return {"accuracy": acc["accuracy"], "f1": f1_score["f1"]}